In [1]:
import openai
import json
import numpy as np
from numpy.linalg import norm
import requests
api_key = open("api_key").read()
openai.api_key = api_key

In [4]:
def get_embedding(text, model="text-embedding-ada-002"):
   # text = text.replace("\n", " ")
   # return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
   url = 'https://api.openai.com/v1/embeddings'
   headers = {
      'Content-Type': 'application/json',
      'Authorization': "Bearer {}".format(api_key)
   }
   data = {
      "input": text,
      "model": model
   }
   res = requests.post(url, headers=headers, json=data)
   res = res.json()
   return res['data'][0]['embedding']

def cos_sim(a, b):
   return np.dot(a, b)/(norm(a)*norm(b))

def save_json(data, filepath=r'new_data.json'):
   with open(filepath, 'w') as fp:
      json.dump(data, fp, indent=4)

In [ ]:
def merge_paragraphs(sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph

In [ ]:
# All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p.json"))
All_articles = json.load(open("data/result/AllTheNews/network/articles.json"))
existing_embeddings = json.load(open("data/raw/AllTheNews/embeddings/2016_10p.json"))

In [ ]:
all_article_ids = list(All_articles.keys())
existing_article_ids = [article['id'] for article in existing_embeddings]
target_article_ids = list(set(all_article_ids) - set(existing_article_ids))
print(len(existing_embeddings), len(all_article_ids), len(target_article_ids))

In [ ]:
# All The News
for article_id in target_article_ids:
    article = All_articles[article_id]
    article['embedding'] = get_embedding(article['summary'])
    existing_embeddings.append({
        "id": article_id,
        "summary": article['summary'],
        "content": article['content'],
        "embedding": article['embedding'],
    })
    print("add: ", len(existing_embeddings))
print(len(existing_embeddings))
# save_json(existing_embeddings, "data/raw/AllTheNews/embeddings/2016_10p.json")

In [ ]:
save_json(existing_embeddings, "data/raw/AllTheNews/embeddings/2016_10p.json")

In [ ]:
All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p.json"))
All_article_embeddings = json.load(open("data/raw/AllTheNews/embeddings/2016_10p.json"))
print(len(All_article_embeddings), len(All_articles))

In [ ]:
server_embeddings = json.load(open("data/result/AllTheNews/network/server/embeddings.json"))
embeddings_dict = { doc['doc_id']: doc for doc in server_embeddings}
frontend_data = json.load(open('data/result/AllTheNews/network/server/frontend_2.json'))
article_node_ids = [node['id'] for node in frontend_data['nodes'] if node['type'] == 'article']
for article_id in article_node_ids:
    if article_id not in embeddings_dict:
        print(article_id)
        # embeddings_dict[article_id] = {
        #     'doc_id': article_id,
        #     'embedding': get_embedding(All_articles[article_id]['summary']),
        # }

In [ ]:
entity_dict = json.load(open('data/result/AllTheNews/network/entities_w_description.json'))
res = {}
count = 0
for entity_id, entity_data in entity_dict.items():
    count += 1
    print("{}/{}".format(count, len(entity_dict)))
    try:
        if entity_data['title'] == entity_data['description']:
            content = entity_data['title']
        else:
            content = entity_data['title'] + ", " + entity_data['description']
        embedding = get_embedding(content)
        res[entity_id] = {
            'id': entity_id,
            'content': content,
            'description': entity_data['description'],
            'embedding': embedding,
        }
    except:
        continue
save_json(res, 'data/result/AllTheNews/network/entities_w_description_embedding.json')

In [6]:
entities_embeddings = json.load(open('data/result/AllTheNews/network/entities_w_description_embedding.json'))
entities = json.load(open('data/result/AllTheNews/network/entities_w_description.json'))
for entity_id, entity_data in entities.items():
    if entity_id not in entities_embeddings:
        if entity_data['title'] == entity_data['description']:
            content = entity_data['title']
        else:
            content = entity_data['title'] + ", " + entity_data['description']
        embedding = get_embedding(content)
        entities_embeddings[entity_id] = {
            'id': entity_id,
            'content': content,
            'description': entity_data['description'],
            'embedding': embedding,
        }
save_json(entities_embeddings, 'data/result/AllTheNews/network/entities_w_description_embedding.json')


In [5]:
server_embeddings = json.load(open("data/result/AllTheNews/network/server/embeddings.json"))
articles = json.load(open("data/result/AllTheNews/preprocessed/2016_10p.json"))
article_dict = { article['id']: article for article in articles}
for article in server_embeddings:
    article_id = article['doc_id']
    article_title = article_dict[article_id]['title']
    article['title'] = article_title
save_json(server_embeddings, "data/result/AllTheNews/network/server/embeddings_w_title.json")
